In [1]:
from random import randint
from random import choice
import copy
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

In [2]:
#Set Up Variables
rows = 3
columns = 3

openMarker = " "
player1Marker = "X"
player2Marker = "O"

In [3]:
#Generate New Game Board
def generateGameBoard(rows, columns):
    gameBoard = []
    
    for r in range(rows):
        gameBoard.append([])
        for c in range(columns):
            gameBoard[r].append(0)
            
    return gameBoard

In [4]:
#Display Current Game Board All Pretty Like
def displayGame(gameBoard):
    for row in range(len(gameBoard)):
        
        if row > 0:
            print("___________")
        
        rowString = " "
        
        for column in range(len(gameBoard[row])):
            
            if column > 0:
                rowString = rowString + " | "
                
            rowString = rowString + str(gameBoard[row][column])
        
        rowString = rowString + " "
        
        print(rowString)
    print("")

In [5]:
def checkOpenSpots(gameBoard):
    openSpots = []
    
    for r in range(len(gameBoard)):
        for c in range(len(gameBoard[r])):
            if gameBoard[r][c] == 0:
                openSpots.append([r, c])
                
    return openSpots

In [6]:
def placeMarker(player, row, column, gameBoard):
    gameBoard[row][column] = player

In [7]:
def checkRowForWin(gameBoard, row, player):
    for column in range(len(gameBoard[row])):
        if gameBoard[row][column] != player:
            return False
                
    return True

In [8]:
def checkColumnForWin(gameBoard, column, player):
    for row in range(len(gameBoard)):
        if gameBoard[row][column] != player:
            return False
                
    return True

In [9]:
def checkDiagForWin(gameBoard, diagonal, player):
    if diagonal == "L":
        for rowColumn in range(len(gameBoard)):
            if gameBoard[rowColumn][rowColumn] != player:
                return False
                
        return True
    else:
        for row in range(len(gameBoard)):
            column = len(gameBoard) - row - 1
            
            if gameBoard[row][column] != player:
                    return False
        
        return True

In [10]:
def checkForWin(gameBoard, player):
    win = False
    
    for row in range(len(gameBoard)):
        win = checkRowForWin(game, row, player)
        
        if win:
            return win
        
    for column in range(len(gameBoard[0])):
        win = checkColumnForWin(game, column, player)
        
        if win:
            return win
        
    win = checkDiagForWin(game, "L", player)
    if win:
        return win
        
    win = checkDiagForWin(game, "R", player)
    if win:
        return win
    
    return win

In [11]:
def checkForTie(gameBoard):
    for row in range(len(gameBoard)):
        for column in range(len(gameBoard[row])):
            if gameBoard[row][column] == 0:
                return False
            
    return True

In [12]:
def playGame(player1, player2, gameBoard, display_game=False):
    gameOver = False
    whoseTurn = randint(1,2)
    if whoseTurn == 1:
        firstMover = 2
    else:
        firstMover = 1
    
    gameLog = []
    current_board = copy.deepcopy(gameBoard)
    gameLog.append(current_board)
    
    player1Moves = 0
    player2Moves = 0
    
    player1Result = "-"
    player2Result = "-"
    
    while not gameOver:
        if whoseTurn == 1:
            whoseTurn = 2
        else:
            whoseTurn = 1
        
        if whoseTurn == 1:
            makeMove(player1, whoseTurn, gameBoard)
            player1Moves = player1Moves + 1
        else:
            makeMove(player2, whoseTurn, gameBoard)
            player2Moves = player2Moves + 1
            
        if display_game:
            displayGame(gameBoard)
        
        current_board = copy.deepcopy(gameBoard)
        gameLog.append(current_board)
        
        if checkForWin(gameBoard, whoseTurn):
            gameOver = True
            
            if whoseTurn == 1:
                player1Result = "W"
                player2Result = "L"
            else:
                player1Result = "L"
                player2Result = "W"
                
        elif checkForTie(gameBoard):
            gameOver = True
            
            player1Result = "T"
            player2Result = "T"
            
    """
    if player1Result == "W":
        print("Player 1 Wins")
    elif player1Result == "T":
        print("It's a Tie")
    else:
        print("Player 2 Wins")
    """
    
    gameData = [player1Result, player2Result, firstMover, player1Moves, player2Moves, gameLog]
    return gameData

In [13]:
def makeMove(player, whoseTurn, gameBoard):
    move = player.make_move(whoseTurn, gameBoard)
    row = move[0]
    column = move[1]
    
    placeMarker(whoseTurn, row, column, gameBoard)

In [16]:
class Tic_Tac_Toe_AI(object):
    def __init__(self, brain="random"):
        self.brain = brain
        self.moves = {"0": (0, 0),
                     "1": (0, 1),
                     "2": (0, 2),
                     "3": (1, 0),
                     "4": (1, 1),
                     "5": (1, 2),
                     "6": (2, 0),
                     "7": (2, 1),
                     "8": (2, 2)}
        self.reverse_moves = {(0, 0): "0",
                     (0, 1): "1",
                     (0, 2): "2",
                     (1, 0): "3",
                     (1, 1): "4",
                     (1, 2): "5",
                     (2, 0): "6",
                     (2, 1): "7",
                     (2, 2): "8"}
    
    def make_move(self, player_number, gameBoard):
        if self.brain == "random":
            move = self.random_move(gameBoard)
        elif self.brain == "dtr":
            move = self.model_move(regressor, player_number, gameBoard)
        
        return move
    
    def get_possible_moves(self, gameBoard):
        return [(r, c) for r, row in enumerate(gameBoard) for c, column in enumerate(row) if column == 0]
    
    def get_move_number(self, gameBoard):
        move_count = 0
        
        for row in gameBoard:
            for column in row:
                if column > 0:
                    move_count += 1
        
        return move_count
    
    def random_move(self, gameBoard):
        possible_moves = self.get_possible_moves(gameBoard)
        move = choice(possible_moves)
        return move
    
    def model_move(self, model, player, gameBoard):
        possible_moves = self.get_possible_moves(gameBoard)
        possible_moves = [self.reverse_moves[move] for move in possible_moves]
        move_number = self.get_move_number(gameBoard)
        moves_with_scores = []
        for possible_move in possible_moves:
            X = np.array([player, move_number] + list(np.ravel(gameBoard)) + [possible_move]).reshape(1, -1)
            move_score = model.predict(X)
            moves_with_scores.append([possible_move, move_score])
        
        move = max(moves_with_scores, key=lambda x: x[1])[0]
        
        return self.moves[move]

In [ ]:
"""
Basic AI Brain:

1. All Options --> The Game Board
2. All Possible Options --> Only Open Spots
3. If only one option ever --> Do it
3. If there is a spot that gives win --> Do it
4. Remove options that would allow the opponent to win next turn
5. No options left or multiple options left --> just random
"""

"""
AI v2:

1. Play Basic AI over and over to create Bayesian?

AI v3:

1. Have 2 random AI play each other many times, recording data for all moves and games
2. Create model from the recorded that scores all possible moves for the AI
3. Pick the move with the highest probability of winning
"""

In [20]:
random_ai_match_results = []

for i in range(1000000):
    game = generateGameBoard(rows, columns)
    ai1 = Tic_Tac_Toe_AI("random")
    ai2 = Tic_Tac_Toe_AI("random")

    results = playGame(ai1, ai2, game)
    random_ai_match_results.append(results)

In [21]:
len(random_ai_match_results)

1000000

In [22]:
Counter([game[0] for game in random_ai_match_results])

Counter({'L': 436567, 'T': 126547, 'W': 436886})

In [25]:
random_ai_match_results[0]

['T',
 'T',
 2,
 4,
 5,
 [[[0, 0, 0], [0, 0, 0], [0, 0, 0]],
  [[0, 2, 0], [0, 0, 0], [0, 0, 0]],
  [[0, 2, 0], [0, 0, 0], [0, 1, 0]],
  [[0, 2, 0], [0, 0, 2], [0, 1, 0]],
  [[0, 2, 0], [0, 1, 2], [0, 1, 0]],
  [[0, 2, 0], [0, 1, 2], [2, 1, 0]],
  [[0, 2, 0], [1, 1, 2], [2, 1, 0]],
  [[0, 2, 0], [1, 1, 2], [2, 1, 2]],
  [[0, 2, 1], [1, 1, 2], [2, 1, 2]],
  [[2, 2, 1], [1, 1, 2], [2, 1, 2]]]]

In [26]:
def expand_single_game_data(single_game_data):
    player1Result, player2Result, firstMover, player1Moves, player2Moves, gameLog = single_game_data
    gameLog = [np.ravel(game_state) for game_state in gameLog]
    
    expanded_data = []
    current_mover = firstMover
    
    for i, game_state in enumerate(gameLog):
        if i < (len(gameLog) - 1):
            move = ""

            for j in range(9):
                if gameLog[i][j] != gameLog[i + 1][j]:
                    move = j
                    break

            if current_mover == 1:
                result = player1Result
            else:
                result = player2Result

            expanded_data.append([current_mover, i] + list(game_state) + [move, result])
            
            if current_mover == 1:
                current_mover = 2
            else:
                current_mover = 1
    
    return expanded_data

In [27]:
expand_single_game_data(random_ai_match_results[0])

[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 'T'],
 [1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 7, 'T'],
 [2, 2, 0, 2, 0, 0, 0, 0, 0, 1, 0, 5, 'T'],
 [1, 3, 0, 2, 0, 0, 0, 2, 0, 1, 0, 4, 'T'],
 [2, 4, 0, 2, 0, 0, 1, 2, 0, 1, 0, 6, 'T'],
 [1, 5, 0, 2, 0, 0, 1, 2, 2, 1, 0, 3, 'T'],
 [2, 6, 0, 2, 0, 1, 1, 2, 2, 1, 0, 8, 'T'],
 [1, 7, 0, 2, 0, 1, 1, 2, 2, 1, 2, 2, 'T'],
 [2, 8, 0, 2, 1, 1, 1, 2, 2, 1, 2, 0, 'T']]

In [48]:
def expand_game_data(raw_game_data):
    intermediate_expanded_data = [expand_single_game_data(game) for game in raw_game_data]
    
    expanded_data = []
    for game in intermediate_expanded_data:
        expanded_data += game
        
    # Convert W, T, L to numeric
    for move in expanded_data:
        if move[-1] == "W":
            move[-1] = 1
        elif move[-1] == "T":
            move[-1] = 0
        else:
            move[-1] = -1
    
    return expanded_data

In [29]:
random_ai_match_results = expand_game_data(random_ai_match_results)

In [31]:
len(random_ai_match_results)

7624742

In [32]:
random_ai_match_results[:20]

[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 7, 0],
 [2, 2, 0, 2, 0, 0, 0, 0, 0, 1, 0, 5, 0],
 [1, 3, 0, 2, 0, 0, 0, 2, 0, 1, 0, 4, 0],
 [2, 4, 0, 2, 0, 0, 1, 2, 0, 1, 0, 6, 0],
 [1, 5, 0, 2, 0, 0, 1, 2, 2, 1, 0, 3, 0],
 [2, 6, 0, 2, 0, 1, 1, 2, 2, 1, 0, 8, 0],
 [1, 7, 0, 2, 0, 1, 1, 2, 2, 1, 2, 2, 0],
 [2, 8, 0, 2, 1, 1, 1, 2, 2, 1, 2, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, -1],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 4, 1],
 [2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 0, 3, -1],
 [1, 3, 0, 0, 0, 2, 1, 0, 0, 2, 0, 0, 1],
 [2, 4, 1, 0, 0, 2, 1, 0, 0, 2, 0, 8, -1],
 [1, 5, 1, 0, 0, 2, 1, 0, 0, 2, 2, 2, 1],
 [2, 6, 1, 0, 1, 2, 1, 0, 0, 2, 2, 1, -1],
 [1, 7, 1, 2, 1, 2, 1, 0, 0, 2, 2, 6, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1],
 [2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 6, 1],
 [1, 2, 0, 1, 0, 0, 0, 0, 2, 0, 0, 7, -1]]

In [33]:
random_ai_match_df = pd.DataFrame(random_ai_match_results, 
                                  columns=["player", "move_number", "sq0", "sq1", "sq2", "sq3", "sq4", "sq5", "sq6", 
                                           "sq7", "sq8", "move", "status"])

In [34]:
random_ai_match_df.head()

,player,move_number,sq0,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,move,status
0,2,0,0,0,0,0,0,0,0,0,0,1,0
1,1,1,0,2,0,0,0,0,0,0,0,7,0
2,2,2,0,2,0,0,0,0,0,1,0,5,0
3,1,3,0,2,0,0,0,2,0,1,0,4,0
4,2,4,0,2,0,0,1,2,0,1,0,6,0


In [35]:
X = random_ai_match_df.drop(["status"], axis=1)
y = random_ai_match_df["status"]

In [36]:
regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=0,
           splitter='best')

In [37]:
print(regressor.predict([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
print(regressor.predict([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]))
print(regressor.predict([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]))
print(regressor.predict([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]))
print(regressor.predict([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4]))
print(regressor.predict([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5]))
print(regressor.predict([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6]))
print(regressor.predict([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7]))
print(regressor.predict([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8]))

[ 0.34785894]
[ 0.20418707]
[ 0.34136733]
[ 0.20062278]
[ 0.49266497]
[ 0.20152366]
[ 0.34287253]
[ 0.19287189]
[ 0.34627954]


/Users/seanosier/anaconda/lib/python3.4/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/seanosier/anaconda/lib/python3.4/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/seanosier/anaconda/lib/python3.4/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a si

In [38]:
game = generateGameBoard(rows, columns)
ai1 = Tic_Tac_Toe_AI("dtr")
ai2 = Tic_Tac_Toe_AI("dtr")

playGame(ai1, ai2, game)

['L',
 'W',
 2,
 3,
 4,
 [[[0, 0, 0], [0, 0, 0], [0, 0, 0]],
  [[0, 0, 0], [0, 2, 0], [0, 0, 0]],
  [[1, 0, 0], [0, 2, 0], [0, 0, 0]],
  [[1, 0, 0], [0, 2, 0], [2, 0, 0]],
  [[1, 0, 1], [0, 2, 0], [2, 0, 0]],
  [[1, 2, 1], [0, 2, 0], [2, 0, 0]],
  [[1, 2, 1], [0, 2, 1], [2, 0, 0]],
  [[1, 2, 1], [0, 2, 1], [2, 2, 0]]]]

In [39]:
initially_trained_ai_match_results = []

for i in range(1000000):
    game = generateGameBoard(rows, columns)
    ai1 = Tic_Tac_Toe_AI("dtr")
    ai2 = Tic_Tac_Toe_AI("dtr")

    results = playGame(ai1, ai2, game)
    initially_trained_ai_match_results.append(results)

In [40]:
len(initially_trained_ai_match_results)

1000000

In [41]:
Counter([game[0] for game in initially_trained_ai_match_results])

Counter({'L': 500872, 'T': 499128})

In [43]:
initially_trained_ai_match_results[0]

['T',
 'T',
 1,
 5,
 4,
 [[[0, 0, 0], [0, 0, 0], [0, 0, 0]],
  [[0, 0, 0], [0, 1, 0], [0, 0, 0]],
  [[2, 0, 0], [0, 1, 0], [0, 0, 0]],
  [[2, 0, 1], [0, 1, 0], [0, 0, 0]],
  [[2, 0, 1], [0, 1, 0], [2, 0, 0]],
  [[2, 0, 1], [1, 1, 0], [2, 0, 0]],
  [[2, 0, 1], [1, 1, 2], [2, 0, 0]],
  [[2, 0, 1], [1, 1, 2], [2, 1, 0]],
  [[2, 2, 1], [1, 1, 2], [2, 1, 0]],
  [[2, 2, 1], [1, 1, 2], [2, 1, 1]]]]

In [44]:
initially_trained_ai_match_results = expand_game_data(initially_trained_ai_match_results)

In [49]:
for move in initially_trained_ai_match_results:
    if move[-1] == "W":
        move[-1] = 1
    elif move[-1] == "T":
        move[-1] = 0
    else:
        move[-1] = -1

In [51]:
initially_trained_ai_match_results[:20]

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0],
 [2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0],
 [2, 3, 2, 0, 1, 0, 1, 0, 0, 0, 0, 6, 0],
 [1, 4, 2, 0, 1, 0, 1, 0, 2, 0, 0, 3, 0],
 [2, 5, 2, 0, 1, 1, 1, 0, 2, 0, 0, 5, 0],
 [1, 6, 2, 0, 1, 1, 1, 2, 2, 0, 0, 7, 0],
 [2, 7, 2, 0, 1, 1, 1, 2, 2, 1, 0, 1, 0],
 [1, 8, 2, 2, 1, 1, 1, 2, 2, 1, 0, 8, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0],
 [2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0],
 [2, 3, 2, 0, 1, 0, 1, 0, 0, 0, 0, 6, 0],
 [1, 4, 2, 0, 1, 0, 1, 0, 2, 0, 0, 3, 0],
 [2, 5, 2, 0, 1, 1, 1, 0, 2, 0, 0, 5, 0],
 [1, 6, 2, 0, 1, 1, 1, 2, 2, 0, 0, 7, 0],
 [2, 7, 2, 0, 1, 1, 1, 2, 2, 1, 0, 1, 0],
 [1, 8, 2, 2, 1, 1, 1, 2, 2, 1, 0, 8, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1],
 [1, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, -1]]

In [52]:
initially_trained_ai_match_df = pd.DataFrame(initially_trained_ai_match_results, 
                                  columns=["player", "move_number", "sq0", "sq1", "sq2", "sq3", "sq4", "sq5", "sq6", 
                                           "sq7", "sq8", "move", "status"])

In [56]:
len(initially_trained_ai_match_df)

7998256

In [53]:
initially_trained_ai_match_df.head()

,player,move_number,sq0,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,move,status
0,1,0,0,0,0,0,0,0,0,0,0,4,0
1,2,1,0,0,0,0,1,0,0,0,0,0,0
2,1,2,2,0,0,0,1,0,0,0,0,2,0
3,2,3,2,0,1,0,1,0,0,0,0,6,0
4,1,4,2,0,1,0,1,0,2,0,0,3,0


In [58]:
X = pd.concat([X, initially_trained_ai_match_df.drop(["status"], axis=1)])
y = pd.concat([y, initially_trained_ai_match_df["status"]])

In [59]:
len(X)

15622998

In [60]:
regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=0,
           splitter='best')

In [64]:
game = generateGameBoard(rows, columns)
ai1 = Tic_Tac_Toe_AI("dtr")
ai2 = Tic_Tac_Toe_AI("dtr")

playGame(ai1, ai2, game)

['L',
 'W',
 2,
 3,
 4,
 [[[0, 0, 0], [0, 0, 0], [0, 0, 0]],
  [[2, 0, 0], [0, 0, 0], [0, 0, 0]],
  [[2, 1, 0], [0, 0, 0], [0, 0, 0]],
  [[2, 1, 0], [0, 0, 0], [0, 0, 2]],
  [[2, 1, 1], [0, 0, 0], [0, 0, 2]],
  [[2, 1, 1], [0, 0, 0], [2, 0, 2]],
  [[2, 1, 1], [1, 0, 0], [2, 0, 2]],
  [[2, 1, 1], [1, 2, 0], [2, 0, 2]]]]